<a href="https://colab.research.google.com/github/TamasCsakvari/BERTrulesOIE/blob/main/BERT_LSOIE_v2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and packages

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.5 MB/s eta 0:00:00


# BERT WITH LSOIE - Self Implemented

## TinyBERT Import and setup

In [2]:
from transformers import AutoModel

model = AutoModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")


Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['cls.predictions.bias', 'fit_denses.2.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'fit_denses.0.weight', 'fit_denses.4.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'fit_denses.2.bias', 'cls.predictions.decoder.weight', 'fit_denses.3.weight', 'fit_denses.4.bias', 'fit_denses.1.weight', 'fit_denses.3.bias', 'fit_denses.0.bias', 'fit_denses.1.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

## LSOIE Dataset Import and exploration

The dataset is in 

In [3]:
import requests
import zipfile
import pandas as pd
from io import BytesIO

url = "https://github.com/Jacobsolawetz/large-scale-oie/raw/master/dataset_creation/lsoie_data/lsoie_data.zip"

response = requests.get(url)

# Make sure the download was successful before continuing
assert response.status_code == 200

# Create a ZipFile object from the downloaded data
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Create an empty dictionary to hold the dataframes
dataframes = {}

# Define column names based on the CoNLL format
column_names = ['ID', 'Token', 'Predicate', 'Sentence_ID', 'Sentence_Token_Number', 
                'Sentence_Length', 'Sentence_Position', 'Argument_Label']

# Iterate through the list of files in the zip file
for filename in zip_file.namelist():
    # Open each file
    with zip_file.open(filename) as f:
        # Read the file into a pandas dataframe and add it to the dictionary
        dataframes[filename] = pd.read_csv(f, sep="\t", header=None, names=column_names)

# Now each file's data is stored as a pandas dataframe in the dictionary. 
# You can access the dataframe for a file like this:
df = dataframes['lsoie_data/lsoie_science_train.conll']

In [4]:
df.head(15)

,ID,Token,Predicate,Sentence_ID,Sentence_Token_Number,Sentence_Length,Sentence_Position,Argument_Label
0,0,Geologists,study,[1],1,1,2,A0-B
1,1,study,study,[1],1,1,2,P-B
2,2,how,study,[1],1,1,2,A1-B
3,3,rocks,study,[1],1,1,2,A1-I
4,4,and,study,[1],1,1,2,A1-I
5,5,minerals,study,[1],1,1,2,A1-I
6,6,form,study,[1],1,1,2,A1-I
7,7,.,study,[1],1,1,2,O
8,0,Geologists,form,[6],6,1,2,O
9,1,study,form,[6],6,1,2,O


In [5]:
df_nl = pd.DataFrame(columns = ["sentence", "subject", "predicate", "object"])

current_Sentence_ID = df.Sentence_ID[0]

for i in range(20):
  print(current_Sentence_ID)
  df_relevant_words = df[df["Sentence_ID"] == current_Sentence_ID]
  


[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


## Applying AutoModelForTokenClassification BERT to the CoNLL data

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_name = "bert-base-cased"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

# Encoding the words and the labels
text = df.groupby("Sentence_ID")["Token"].apply(list).tolist()
tags = df.groupby("Sentence_ID")["Argument_Label"].apply(list).tolist()

tag_values = list(set(df["Argument_Label"].values))
tag2id = {t: i for i, t in enumerate(tag_values)}

encodings = tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
labels = encode_tags(tags, encodings)

# Preparing the final dataset
encodings["labels"] = label

TypeError: ignored

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SRLDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(item['labels'])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

dataset = SRLDataset(encodings)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


####

from transformers import AdamW

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
    total_loss = 0
    model.train()

    for batch in dataloader:
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {total_loss}")



Experimenting with formatting the data:

In [ ]:
# Create an empty list to store the tuples
tuples = []

# Group the dataframe by Sentence_ID
grouped = df.groupby('Sentence_ID')

# Iterate over the groups
for name, group in grouped:
    # Create a tuple for each group and add it to the list
    tuples.append(tuple(group['Token'].values))

# Now 'tuples' is a list of tuples, where each tuple contains all the tokens of one information unit.
for t in tuples[:10]:
    print(t)

('A', 'reused', 'item', 'can', 'be', 'used', 'in', 'the', 'same', 'way', 'by', 'someone', 'else', '.', 'Well', 'go', 'out', 'on', 'the', 'research', 'vessel', '-LRB-', 'R/V', 'in', 'ship-speak', '-RRB-', 'Atlantis', ',', 'owned', 'by', 'the', 'US', 'Navy', 'and', 'operated', 'by', 'the', 'Woods', 'Hole', 'Oceanographic', 'Institution', 'for', 'the', 'oceanographic', 'community', '.', 'Might', 'a', 'similar', 'ball', 'break', 'up', 'more', 'quickly', ',', 'or', 'be', 'unable', 'to', 'spin', 'as', 'fast', '?')
('Studying', 'the', 'layers', 'of', 'rock', 'beneath', 'the', 'surface', 'helps', 'us', 'to', 'understand', 'the', 'history', 'of', 'planet', 'Earth', '-LRB-', 'Figure', '1.12', '-RRB-', '.', 'Trapped', 'heat', 'causes', 'the', 'atmosphere', 'to', 'heat', 'up', '.', 'Using', 'radars', 'and', 'satellites', ',', 'meteorologists', 'work', 'to', 'predict', ',', 'or', 'forecast', ',', 'the', 'weather', '-LRB-', 'Figure', '1.14', '-RRB-', '.', 'Flowing', 'water', 'is', 'a', 'very', 'impo

# BERT from Adam Kovacs


## Import packages

In [ ]:
%%capture
!pip install transformers

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification# We only want to finetune the classification layer on top of BERT
import pandas as pd

from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import numpy as np


SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Donwload and transform data

### Data transformation functions

In [ ]:
def read_dataset():
    train_data = pd.read_csv("./data/offenseval.tsv", sep="\t")
    return train_data

In [ ]:
def add_binary_label(train_data):
    labels = {"NOT": 0, "OFF": 1}

    train_data["label"] = [labels[item] for item in train_data.subtask_a]
    train_data["tweet"] = train_data["tweet"].str.replace("@USER", "")

    return train_data

In [ ]:
def split_data(train_data, random_seed):
    tr_data, val_data = train_test_split(train_data, test_size=0.3, random_state=SEED)
    return tr_data, val_data

### Unknown functions:P

In [ ]:
def prepare_dataloader_with_padding(tr_data, val_data, word_to_ix):
    # First create the id representations of the input vectors
    # Then pad the sequences so all of the input is the same size
    # We padded texts for the whole dataset, this could have been done batch-wise also!
    tr_data_vecs = pad_sequence(
        create_input(tr_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True,
        padding_value=3001,
    )
    tr_labels = torch.LongTensor(tr_data.label.tolist()).to(device)
    tr_lens = torch.LongTensor(
        [len(i) for i in create_input(tr_data.tweet, an, word_to_ix.vocabulary_)]
    )

    # We also add the texts to the batches
    # This is for the Transformer models, you wont need this in the next experiments
    tr_sents = tr_data.tweet.tolist()

    val_data_vecs = pad_sequence(
        create_input(val_data.tweet, an, word_to_ix.vocabulary_),
        batch_first=True,
        padding_value=3001,
    )
    val_labels = torch.LongTensor(val_data.label.tolist()).to(device)
    val_lens = torch.LongTensor(
        [len(i) for i in create_input(val_data.tweet, an, word_to_ix.vocabulary_)]
    )

    val_sents = val_data.tweet.tolist()

    tr_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            tr_data_vecs, tr_labels, tr_lens, tr_sents
        )
    ]
    val_data_loader = [
        (sample, label, length, sent)
        for sample, label, length, sent in zip(
            val_data_vecs, val_labels, val_lens, val_sents
        )
    ]

    return tr_data_loader, val_data_loader

In [ ]:
def create_input(dataset, analyzer, vocabulary):
    dataset_as_indices = []

    # We go through each tweet in the dataset
    # We need to add two additional symbols to the vocabulary
    # We have 3000 features, ranged 0-2999
    # We add 3000 as an id for the "unknown" words not among the features
    # 3001 will be the symbol for padding, but about this later!
    for tweet in dataset:
        tokens = analyzer(tweet)
        token_ids = []

        for token in tokens:
            # if the token is in the vocab, we add the id
            if token in vocabulary:
                token_ids.append(vocabulary[token])
            # else we add the id of the unknown token
            else:
                token_ids.append(3000)

        # if we removed every token during preprocessing (stopword removal, lemmatization), we add the unknown token to the list so it won't be empty
        if not token_ids:
            token_ids.append(3000)
        dataset_as_indices.append(torch.LongTensor(token_ids).to(device))

    return dataset_as_indices

### Download and transform data

In [ ]:
import os

if not os.path.isdir("./data"):
    os.mkdir("./data")

import urllib.request

u = urllib.request.URLopener()
u.retrieve(
    "https://raw.githubusercontent.com/ZeyadZanaty/offenseval/master/datasets/training-v1/offenseval-training-v1.tsv",
    "data/offenseval.tsv",
)

('data/offenseval.tsv', <http.client.HTTPMessage at 0x7fd07d76f460>)

In [ ]:
train_data_unprocessed = read_dataset()
train_data_unprocessed

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [ ]:
train_data = add_binary_label(train_data_unprocessed)

In [ ]:
tr_data, val_data = split_data(train_data, SEED)

In [ ]:
# We add the length of the tweets so sentences with similar lengths will be next to each other
# This can be important because of padding
tr_data["length"] = tr_data.tweet.str.len()
val_data["length"] = val_data.tweet.str.len()

tr_data = tr_data.sort_values(by="length")
val_data = val_data.sort_values(by="length")

KeyError: ignored

In [ ]:
tr_data_loader, val_data_loader = prepare_dataloader_with_padding(
    tr_data, val_data, word_to_ix
)

train_iterator, valid_iterator = create_dataloader_iterators_with_padding(
    tr_data_loader, val_data_loader, BATCH_SIZE
)

NameError: ignored

## Prepare model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
_ = model.to(device)

for p in model.base_model.parameters():
    p.requires_grad = False

params = list(model.named_parameters())print(f"The BERT model has {len(params)} different named parameters.")print("==== Embedding Layer ====\n")for p in params[0:5]:    print(f"{p[0]} {str(tuple(p[1].size()))}")print("\n==== First Transformer ====\n")for p in params[5:21]:    print(f"{p[0]} {str(tuple(p[1].size()))}")print("\n==== Output Layer ====\n")for p in params[-4:]:    print(f"{p[0]} {str(tuple(p[1].size()))}")

N_EPOCHS = 5
optimizer = optim.Adam(model.parameters())

SyntaxError: ignored

## Train model

In [ ]:
for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_epoch_loss = 0
    train_epoch_prec = 0
    train_epoch_recall = 0
    train_epoch_fscore = 0
    model.train()

    # We use our own iterator but now use the raw texts instead of the ID tokens
    for train_batch in train_iterator:
        labels = train_batch[1]
        texts = train_batch[3]

        optimizer.zero_grad()

        # We use BERT's own tokenizer on raw texts
        # Check the documentation: https://huggingface.co/transformers/main_classes/tokenizer.html
        encoded = tokenizer(
            texts,
            truncation=True,
            max_length=128,
            padding=True,
            return_tensors="pt",
        )

        # BERT converts texts into IDs of its own vocabulary
        input_ids = encoded["input_ids"].to(device)
        # Mask to avoid performing attention on padding token indices.
        attention_mask = encoded["attention_mask"].to(device)

        # Run the model
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0]
        predictions = outputs[1]
        prec, recall, fscore = calculate_performance(predictions, labels)

        loss.backward()
        optimizer.step()

        train_epoch_loss += loss.item()
        train_epoch_prec += prec.item()
        train_epoch_recall += recall.item()
        train_epoch_fscore += fscore.item()

    train_loss = train_epoch_loss / len(train_iterator)
    train_prec = train_epoch_prec / len(train_iterator)
    train_rec = train_epoch_recall / len(train_iterator)
    train_fscore = train_epoch_fscore / len(train_iterator)

    # And validate your model on the validation set
    valid_epoch_loss = 0
    valid_epoch_prec = 0
    valid_epoch_recall = 0
    valid_epoch_fscore = 0
    model.eval()

    with torch.no_grad():
        for valid_batch in valid_iterator:
            labels = valid_batch[1]
            texts = valid_batch[3]

            encoded = tokenizer(
                texts,
                truncation=True,
                max_length=128,
                padding=True,
                return_tensors="pt",
            )
            input_ids = encoded["input_ids"].to(device)
            attention_mask = encoded["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            predictions = outputs[1]
            prec, recall, fscore = calculate_performance(predictions, labels)

            # We add batch-wise loss to the epoch-wise loss
            valid_epoch_loss += loss.item()
            valid_epoch_prec += prec.item()
            valid_epoch_recall += recall.item()
            valid_epoch_fscore += fscore.item()

    valid_loss = valid_epoch_loss / len(valid_iterator)
    valid_prec = valid_epoch_prec / len(valid_iterator)
    valid_rec = valid_epoch_recall / len(valid_iterator)
    valid_fscore = valid_epoch_fscore / len(valid_iterator)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(
        f"\tTrain Loss: {train_loss:.3f} | Train Prec: {train_prec*100:.2f}% | Train Rec: {train_rec*100:.2f}% | Train Fscore: {train_fscore*100:.2f}%"
    )
    print(
        f"\t Val. Loss: {valid_loss:.3f} |  Val Prec: {valid_prec*100:.2f}% | Val Rec: {valid_rec*100:.2f}% | Val Fscore: {valid_fscore*100:.2f}%"
    )